Leitura e visualização das primeiras linhas do dataset.

In [19]:
import pandas as pd

url = 'https://raw.githubusercontent.com/alura-cursos/combina-classificadores/main/dados/bank_additional_full_processed.csv'
df = pd.read_csv(url, sep=',')


df.head()


,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,1,0,0,0,1,0,0,0,1,0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,1,0,0,0,1,0,0,0,1,0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,1,0,0,0,1,0,0,0,1,0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,1,0,0,0,1,0,0,0,1,0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,1,0,0,0,1,0,0,0,1,0


Instalação e importação das bibliotecas necessárias

In [20]:
!pip install mlxtend

In [21]:
!pip install catboost

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from mlxtend.classifier import StackingCVClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

Parâmetros de busca para o grid_search

In [23]:
parametros = {'bootstrap': False,
 'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 6,
 'n_estimators': 200}

In [24]:
# Separação das features e do alvo
X = df.drop('y', axis=1)
y = df['y']

# Divisão em treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

# Classificadores de base
catboost = CatBoostClassifier(verbose=0)  # desligando a saída de treino
extratrees = ExtraTreesClassifier(**parametros,random_state=42)
gaussiannb = GaussianNB()

scaler = StandardScaler()
scaler.fit(X_treino)
X_treino = scaler.transform(X_treino)
X_teste = scaler.transform(X_teste)
# Classificador Meta
logistic = LogisticRegression()

# Usando o StackingCVClassifier
stack = StackingCVClassifier(classifiers=[catboost, extratrees, gaussiannb],
                             meta_classifier=logistic,
                             cv=5,
                             random_state=42,
                             verbose=1,
                             n_jobs=-1)



# Ajustando o modelo aos dados de treino
stack.fit(X_treino, y_treino)



Fitting 3 classifiers...
Fitting classifier1: catboostclassifier (1/3)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.4min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting classifier2: extratreesclassifier (2/3)


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   32.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting classifier3: gaussiannb (3/3)


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


StackingCVClassifier(classifiers=[<catboost.core.CatBoostClassifier object at 0x7bbe37faf700>,
                                  ExtraTreesClassifier(min_samples_split=6,
                                                       n_estimators=200,
                                                       random_state=42),
                                  GaussianNB()],
                     cv=5, meta_classifier=LogisticRegression(), n_jobs=-1,
                     random_state=42, verbose=1)

In [25]:
# Usando o `pipeline` para fazer previsões.
y_pred = stack.predict(X_teste)

In [26]:
stack.score(X_teste,y_teste)

0.9166059723233795